In [ ]:
import warnings
warnings.simplefilter('ignore')

In [ ]:
%matplotlib widget

In [ ]:
import ipympl
import matplotlib
import matplotlib.pylab as plt

In [ ]:
import vaex
import vaex.jupyter
import ipywidgets as widgets

In [ ]:
import sys
import os

In [ ]:
use_s3 = os.environ.get('USE_S3', '') == '1'
if use_s3:
    df = vaex.open('s3://astrosurveydata/gaia_ps1_nochunk.hdf5?cache=true')
else:
    with open('vaex_token_trusted.txt') as f:
        token = f.read().strip()
    df = vaex.open(f'ws://ec2-18-222-183-211.us-east-2.compute.amazonaws.com:9000/gaia_ps1_nochunk?token_trusted={token}')[:1_000_000]

In [ ]:
# df[:]

In [ ]:
limits_g = [12, 22]
shape_g = 100

In [ ]:
# df.select(df.phot_g_mean_mag==df.phot_g_mean_mag, name='visual')
# df.select(df.phot_g_mean_mag==df.phot_g_mean_mag, name='custom')
# df.select('visual & custom')

In [ ]:
use_visual_filter = use_expression_filter = True
visual_filter = (df.phot_g_mean_mag >= 18) & (df.phot_g_mean_mag <= 19)
custom_filter = None

In [ ]:
# plt.ioff()
plt.ion()

In [ ]:
limits_l = [360, 0]
limits_b = [-90, 90]
fig_sky = plt.figure()
ax_sky = plt.gca()

# see https://github.com/matplotlib/jupyter-matplotlib/issues/127
fig_sky.canvas.layout.min_width = '200px'
fig_sky.canvas.layout.min_height = '200px'

df.plot(df.l, df.b, limits=[limits_l, limits_b]);

In [ ]:
limits_l, limits_b

In [ ]:
# based on https://stackoverflow.com/questions/31490436/matplotlib-finding-out-xlim-and-ylim-after-zoom
def on_xlims_change(axes):
    global limits_l
    print('x change')
    limits_l = ax_sky.get_xlim()
    update_plots()
ax_sky.callbacks.connect('xlim_changed', on_xlims_change)

def on_ylims_change(axes):
    global limits_b
    print('y change')
    limits_b = ax_sky.get_ylim()
    update_plots()
ax_sky.callbacks.connect('ylim_changed', on_ylims_change);

In [ ]:
ax_sky

In [ ]:
fig_sky.gca()

In [ ]:
ax_sky.get_xlim()

In [ ]:
fig_mag = plt.figure()
# see https://github.com/matplotlib/jupyter-matplotlib/issues/127
fig_mag.canvas.layout.min_width = '200px'
fig_mag.canvas.layout.min_height = '200px'
plt.title('Select a G mag range')
artists_mag = df.plot1d(df.phot_g_mean_mag, limits=limits_g, shape=shape_g)
fig_mag.canvas.toolbar_visible = False
def f(vmin, vmax):
    global visual_filter
    values = [vmin, vmax]
    vmin, vmax = min(values), max(values)
    visual_filter = (df.phot_g_mean_mag >= vmin) & (df.phot_g_mean_mag <= vmax)
    update_plots()
axes = fig_mag.axes[0]
mpl_widget = matplotlib.widgets.SpanSelector(axes, f, 'horizontal', useblit=True)

In [ ]:
progress = widgets.FloatProgress(min=0, max=1, description='done')
progress

In [ ]:
def update_progress(value):
    progress.value = value

In [ ]:
# @vaex.jupyter.utils.interactive_selection(df)

@vaex.jupyter.debounced(0.5)
def update_plots():
    global mpl_widget  # if we don't keep a global ref to this widget, it gets garbage collected
    print('update')
    
    plt.figure(fig_mag.number)
    plt.clf()
    plt.title('Select a G mag range')
    progress.description = 'mag'
    artists = df.plot1d(df.phot_g_mean_mag, limits=limits_g, shape=shape_g, progress=update_progress)
    artists_mag.extend(artists)
    dff = df
    if visual_filter and use_visual_filter:
        dff = dff[visual_filter]
    if custom_filter and use_expression_filter:
        dff = dff[custom_filter]
    progress.description = 'mag filtered'
    dff.plot1d(dff.phot_g_mean_mag, limits=limits_g, shape=shape_g, color='green', progress=update_progress)
    plt.draw()
    # since we clear the figure, we also need to recreate the selector widget
    axes = fig_mag.axes[0]
    mpl_widget = matplotlib.widgets.SpanSelector(axes, f, 'horizontal', useblit=True)

    plt.figure(fig_sky.number)   
    fig_sky.axes[1].remove()  # remove colorbar
    plt.sca(ax_sky)
    ax_sky.clear()

    if visual_filter is not None or custom_filter is not None:
        progress.description = 'sky ref'
        count_ref = df.count(binby=[df.l, df.b], limits=[limits_l, limits_b], progress=update_progress)
        progress.description = 'sky filtered'
        count_filter = dff.count(binby=[dff.l, dff.b], limits=[limits_l, limits_b], progress=update_progress)
        ratio = count_filter / count_ref
        df.plot(df.l, df.b, grid=ratio, limits=[limits_l, limits_b])
    else:
        df.plot(df.l, df.b, limits=[limits_l, limits_b])

    progress.description = 'done'
    # since we clear the figure, we also need to reconnect to the axes events
#     listen_to_sky_axes()
    plt.draw()

update_plots()

In [ ]:
default_custom_selection = """(phot_g_mean_mag > 10) &\\
(phot_g_mean_mag < 100)
"""

In [ ]:

custom_selection_widget = widgets.Textarea(description='Selection', value=default_custom_selection)
custom_selection_widget.layout.width = '800px'
custom_selection_widget.layout.height = '200px'
custom_selection_widget

In [ ]:
error_msg_widget = widgets.HTML()
error_msg_widget

In [ ]:
def update_custom_selection(change):
    global custom_filter
    try:
        df.validate_expression(change.new)
    except Exception as e:
        error_msg_widget.value = f'<span style="color: red;">{e}</span>'
    error_msg_widget.value = ''
    custom_filter = df[change.new]
    update_plots()
custom_selection_widget.observe(update_custom_selection, 'value')

In [ ]:
custom_selection_widget.continuous_update = True

In [ ]:
import ipyvuetify as v

In [ ]:
custom_selection_widget_vuetify = v.Textarea(v_model=default_custom_selection,
                                             label="Filter by custom expression",
                                             prepend_icon='filter_list',
                                             placeholder="e.g.: \n" + default_custom_selection)
custom_selection_widget_vuetify

In [ ]:
def update_custom_selection(change):
    global custom_filter
    try:
        df.validate_expression(change.new)
    except Exception as e:
        tf.error_messages = str(e)
        return
    custom_selection_widget_vuetify.error_messages = None
    custom_filter = df[change.new]
    update_plots()
custom_selection_widget_vuetify.observe(update_custom_selection, 'v_model')

In [ ]:
progress.layout.width = '95%'
progress.style.bar_color = '#b3d4fc'

In [ ]:
v.Layout(
    _metadata={'mount_id': 'content-main'},
    row=True, wrap=True, align_center=True, children=[
    v.Flex(xs12=True, children=[
        progress
    ]),
    v.Flex(xs12=True, lg6=True, children=[
        widgets.VBox([fig_mag.canvas])
    ]),
    v.Flex(xs12=True, lg6=True, children=[
        widgets.VBox([fig_sky.canvas])
    ]),
    v.Flex(xs12=True, lg6=True, children=[
        custom_selection_widget_vuetify
    ]),
])


In [ ]:
widget_use_visual_filter = v.Switch(label='Use visual filter', v_model=True)
widget_use_expression_filter = v.Switch(label='Use expression filter', v_model=True)

In [ ]:
v.Layout(
    _metadata={'mount_id': 'content-nav'},
    pa_2=True, column=True, children=[
        widget_use_visual_filter,
        widget_use_expression_filter
    ]
)


In [ ]:
def toggle_use_filter(change):
    global use_visual_filter, use_expression_filter
    use_visual_filter = widget_use_visual_filter.v_model
    use_expression_filter = widget_use_expression_filter.v_model
    update_plots()
widget_use_visual_filter.observe(toggle_use_filter, 'v_model')
widget_use_expression_filter.observe(toggle_use_filter, 'v_model')

In [ ]:
widget_use_expression_filter.v_model